<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [ ]:
# !pip install konlpy

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 98.8 MB/s eta 0:00:00


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

In [ ]:
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/project_final/data/"
DATA_PATH = "C:\python-code\project_final\data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 데이터 불러오기

### 전체 데이터셋

In [ ]:
yo_review = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['고객리뷰']
yo_reply = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['사장댓글']
yo_star = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['별점']

train = pd.concat([yo_review, yo_reply, yo_star], axis=1)
rename_cols = ['review', 'reply', 'star']
train.columns = rename_cols
train['star'].value_counts().sort_index() # 별점 분포 확인

# shuffled_df = train.sample(frac=1, random_state=2)  # frac=1은 모든 행을 선택, random_state는 재현 가능한 랜덤 설정
# train = shuffled_df.iloc[:1000].reset_index(drop=True)

star
1       92
2       64
3      179
4      560
5    17791
Name: count, dtype: int64

In [ ]:
# 중복된 답변을 한 경우 행 전체 삭제
train.drop_duplicates(subset='reply', keep=False, inplace=True)

## 데이터 정제

- 불필요한 문자 삭제

In [ ]:
review_list = []
for review in train['review']:
    if isinstance(review, str):
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

,review,reply,star
1,리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...,5
2,잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...,5
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...,5
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...,5
5,양많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다ㅠㅠ😭\n\n고객님들의 배고픔...,5


In [ ]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

,review,reply,star
1,리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,5
2,잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다 저희를 찾아 주신 만큼 1...,5
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다 저희 매장의 메뉴를 제대로 즐기신 거 ...,5
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,5
5,양많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지 감동입니다 고객님들의 배고픔을 저희는 지...,5


### "~~님" → "고객님"으로 변환

In [ ]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)

# 텍스트 Paraphrase
```
https://huggingface.co/lcw99/t5-base-korean-paraphrase
```

In [ ]:
train['star'].value_counts().sort_index() # 별점 분포 확인

star
1      90
2      61
3     153
4     422
5    9776
Name: count, dtype: int64

In [ ]:
train[train['star'] == 1].index

Index([   96,   144,   252,   363,   401,   658,  1552,  1835,  1951,  1988,
        2002,  2049,  2171,  2228,  2229,  2509,  2653,  2780,  2858,  3636,
        3731,  3861,  4390,  4857,  4922,  4990,  5120,  5662,  6005,  6984,
        7367,  7427,  7450,  7508,  7653,  7693,  7849,  8068,  8238,  8294,
        8477,  8555,  8605,  8652,  8790,  9233,  9449,  9594,  9799,  9881,
       10398, 10811, 10938, 11612, 11670, 12519, 12526, 12583, 12646, 12755,
       13079, 13135, 13173, 13209, 13347, 13714, 13980, 14404, 14410, 14413,
       14425, 14515, 14599, 14616, 14853, 15130, 15299, 15599, 15798, 15800,
       15811, 15911, 16088, 16113, 16265, 16302, 16401, 17313, 17679, 18188],
      dtype='int64')

In [ ]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

In [ ]:
train['review'][96]

'리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다 배달이 20분 넘게 안오길래 전화해서 배달이 안온다 했더니 여자분께서 한숨 푹 쉬시면서 하아 비가 와서 배달 기사 안 잡힌다 하시는데 제가 뭐 그런거에 이해 못하는 사람도 아니고 그냥 안와서 물어보는건데 한숨음 왜 크게 쉬시나요 아니 돈을 내고 배달 시간도 20분이나 지나서 물어보는게 잘못된건가요 배달기사한테 도착하면 문자 달라고 하니까 문자도 안주셔서 밥은 식은 상태로 먹고 진짜 배달 시켜 먹어본곳 중에 제일 최악이네요 여긴'

In [ ]:
prompt = train['review'][96]
generated_text = pipe(prompt, max_length=128, num_beams=3, num_return_sequences=1, no_repeat_ngram_size=3, top_k=50, top_p=0.95, early_stopping=True)
# print("리뷰: ", prompt)
print("paraphrase: ", generated_text[0]['generated_text'])

paraphrase:  아니 그런데 배달이 20분 넘게 안와서 전화해서 물어보니 배달 기사님이 비가 와서 못 간다고 하시는데 제가 그 말을 이해할 수 없습니다.


In [ ]:
train['reply'][96]

'안녕하세요 1988응답하라추억의옛날도시락 입니다 주신 리뷰 잘 읽었습니다 뒤돌아 보니 저희 응대에 미흡했던 점이 있던 것 같네요 응대 메뉴얼을 재점검 하도록 하겠습니다 이미 느끼신 불편함을 주워 담을 순 없지만 다음에 오실 때 요청 사항에 이 내용 남겨 주시면 저희가 더 신경 쓸 수 있도록 하겠습니다 그땐 온전한 만족감을 드릴 수 있도록 최선을 다하겠습니다 감사합니다 '

In [ ]:
prompt = train['reply'][96]
generated_text = pipe(prompt, max_length=128, num_beams=4, num_return_sequences=1, no_repeat_ngram_size=4, top_k=50, top_p=0.95, early_stopping=True)
print(generated_text[0]['generated_text'])

저희가 미처 생각하지 못한 부분이 있기 때문에 다시 한번 감사의 마음을 전하고 싶습니다.


In [ ]:
train['review'][144]

'진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고'

In [ ]:
prompt = train['review'][144]
generated_text = pipe(prompt, max_length=128, num_beams=5, num_return_sequences=5, no_repeat_ngram_size=3, top_k=50, top_p=0.95, early_stopping=True)
# print("리뷰: ", prompt)
print("paraphrase: ", generated_text[0]['generated_text'])

paraphrase:  진짜 2만원씩 내고 두번먹을 양이면 충분한데 제가 장난으로 글을 쓴것도 아니고 진짜 양이 너무 많아서 남길 생각도 없습니다.


In [ ]:
train['reply'][144]

'솔직한 리뷰 남겨 주셔서 감사합니다 저희 1988응답하라추억의옛날도시락은 언제나 약속 드린 정량을 드리기 위해 노력하고 있습니다 드렸을 때 준비 과정을 다시 살펴 보고 혹시 미흡했던 점은 없었는지 확인하겠습니다 아울러 본사 측에도 이 의견 전달 해서 더 개선 될 수 있는 부분은 없는지 검토해 보겠습니다 다음에 오실 때 요청 사항에 이 내용 남겨 주시면 저희가 더 신경 쓸 수 있도록 하겠습니다 감사합니다 '

In [ ]:
prompt = train['reply'][144]
generated_text = pipe(prompt, max_length=128, num_beams=4, num_return_sequences=1, no_repeat_ngram_size=4, top_k=50, top_p=0.95, early_stopping=True)
print(generated_text[0]['generated_text'])

저희 1988응답하라추억의옛날도시락은 언제나 약속 드린 정량을 지키기 위해 노력하고 있습니다 전달 시 준비 과정을 다시 살펴 보고 혹시 부족한 점은 없는지 검토해 보겠습니다.


## 하이퍼파라미터 튜닝

In [ ]:
star_1 = train[train['star'] == 1]
star_2 = train[train['star'] == 2]
star_3 = train[train['star'] == 3]
star_4 = train[train['star'] == 4]

train_star = pd.concat([star_1, star_2, star_3, star_4], ignore_index=True)
train_star

,review,reply,star
0,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다 배달이 20분 넘게 안오길래 전...,안녕하세요 1988응답하라추억의옛날도시락 입니다 주신 리뷰 잘 읽었습니다 뒤돌아 보...,1
1,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고,솔직한 리뷰 남겨 주셔서 감사합니다 저희 1988응답하라추억의옛날도시락은 언제나 약...,1
2,만들어 놓은지 한참 된 커피 배달 예정 시간 25분이 지났어요,안녕하세요 배달이 늦어져 죄송합니다 한파로 길이 얼어서 고객님들이 상황이 여유치가 ...,1
3,밥이 쉬었어요 단무지 때문도 아니고 김이 많아서 그런것도 아니고 쉬어서 냄새가 나서...,밥이 쉬다뇨 말도 안되는 소리좀 하지마세요 진짜 먼 정신이 이상한 사람인가 하루에 ...,1
4,잠실점에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와보니...,요기요가 주문어플에 렉이 자주 있어서 여러가지로 혼돈이 있어 정말 죄송하게 되었습니...,1
...,...,...,...
721,맛있게 잘 먹었습니다,맛있게 드셔주셔서 감사드립니다 더 노력하는 보둘미역이 되겠습니다,4
722,국은 시원하게 맛있었어요 국물양이 더 많았으면 더 좋았겠습니다,맛있게 드셨다니 감사드립니다 매장과 같이 최대한 드리고있습니다 배달용기에 국물을 더...,4
723,배송 맛 반찬등 구성도 좋고 맛도 괜찮고 가격도 넘좋아요 양이 좀만더 많음 좋겠어용...,양이 좀 적어셨나봐요 보통 1인분에 두명도 드실수있는 양인데 그래도 맛있게 드셨다니...,4
724,잘먹었숩당 양은 증말로 많아요,맛있게 드셔주셔서 감사합니다 별 5개 받도록 노력하겠습니다,4


### BLEU 계산 함수

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

### review

In [ ]:
# import optuna
# from transformers import pipeline

# def objective(trial):
#     hp = {
#         "num_beams" : trial.suggest_int("num_beams", 1, 10),
#         "num_return_sequences" : trial.suggest_int("num_return_sequences", 1, 5),
#         "no_repeat_ngram_size" : trial.suggest_int("no_repeat_ngram_size", 1, 5),
#         "top_k" : trial.suggest_int("top_k", 10, 100),
#         "top_p" : trial.suggest_float("top_p", 0.8, 1.0)
#     }
#     # num_return_sequences가 num_beams보다 작거나 같아야 하기 때문에 조건문 추가
#     if hp["num_return_sequences"] > hp["num_beams"]:
#         hp["num_return_sequences"] = hp["num_beams"]

#     pipe = pipeline(
#         "text2text-generation",
#         model="lcw99/t5-base-korean-paraphrase",
#         device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     )

#     bleu_scores = []
#     for prompt in tqdm(train_star["review"]): # star1~4까지
#         generated_text = pipe(prompt, max_length=128, **hp, early_stopping=True)

#         bleu_score = calculate_bleu(prompt, generated_text[0]['generated_text'])
#         bleu_scores.append(bleu_score)

#     # 모든 입력 텍스트에 대한 BLEU 점수 평균 계산 및 반환
#     bleu_score_avg = sum(bleu_scores) / len(bleu_scores)
#     return bleu_score_avg

In [ ]:
# # 하이퍼파라미터 최적화
# sampler = optuna.samplers.TPESampler(seed=SEED)
# study = optuna.create_study(sampler=sampler, direction="maximize")
# study.optimize(objective, n_trials=12)

# # 최적 값 출력
# best_bleu_score = study.best_value
# best_params = study.best_params
# print("best_bleu_score:", best_bleu_score)
# print("best_params:", best_params)

[I 2023-09-04 18:21:25,156] A new study created in memory with name: no-name-88447d6d-9e54-42fa-ad15-c88fcd87f12d


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-04 18:57:51,133] Trial 0 finished with value: 0.285731536841964 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}. Best is trial 0 with value: 0.285731536841964.


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-04 19:25:28,423] Trial 1 finished with value: 0.28140373147755066 and parameters: {'num_beams': 2, 'num_return_sequences': 1, 'no_repeat_ngram_size': 5, 'top_k': 64, 'top_p': 0.941614515559209}. Best is trial 0 with value: 0.285731536841964.


...

  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-04 23:49:39,909] Trial 10 finished with value: 0.285731536841964 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 11, 'top_p': 0.8030735474656209}. Best is trial 0 with value: 0.285731536841964.


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-05 00:20:59,956] Trial 11 finished with value: 0.285731536841964 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 17, 'top_p': 0.8007473955626367}. Best is trial 0 with value: 0.285731536841964.


best_bleu_score: 0.285731536841964
best_params: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}


In [ ]:
review_best_params = {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}

over_review = []
for prompt in train_star['review']:
    generated_text = pipe(prompt, max_length=128, **review_best_params, early_stopping=True)
    generated_text = generated_text[0]['generated_text']
    over_review.append(generated_text)

### reply

In [ ]:
import optuna
from transformers import pipeline

def objective(trial):
    hp = {
        "num_beams" : trial.suggest_int("num_beams", 1, 10),
        "num_return_sequences" : trial.suggest_int("num_return_sequences", 1, 5),
        "no_repeat_ngram_size" : trial.suggest_int("no_repeat_ngram_size", 1, 5),
        "top_k" : trial.suggest_int("top_k", 10, 100),
        "top_p" : trial.suggest_float("top_p", 0.8, 1.0)
    }
    # num_return_sequences가 num_beams보다 작거나 같아야 하기 때문에 조건문 추가
    if hp["num_return_sequences"] > hp["num_beams"]:
        hp["num_return_sequences"] = hp["num_beams"]

    pipe = pipeline(
        "text2text-generation",
        model="lcw99/t5-base-korean-paraphrase",
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    )

    bleu_scores = []
    for prompt in tqdm(train_star["reply"]): # star1~4까지
        generated_text = pipe(prompt, max_length=128, **hp, early_stopping=True)

        bleu_score = calculate_bleu(prompt, generated_text[0]['generated_text'])
        bleu_scores.append(bleu_score)

    # 모든 입력 텍스트에 대한 BLEU 점수 평균 계산 및 반환
    bleu_score_avg = sum(bleu_scores) / len(bleu_scores)
    return bleu_score_avg

In [ ]:
# 하이퍼파라미터 최적화
sampler = optuna.samplers.TPESampler(seed=SEED)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=12)

# 최적 값 출력
best_bleu_score = study.best_value
best_params = study.best_params
print("best_bleu_score:", best_bleu_score)
print("best_params:", best_params)

[I 2023-09-05 09:06:47,480] A new study created in memory with name: no-name-d76cd6a4-4637-44d8-9c4e-b723d6953cec


  0%|          | 0/726 [00:00<?, ?it/s]

# 오버샘플링

In [ ]:
# 별점 분포 확인
star_distribution = train['star'].value_counts().sort_index()
print(star_distribution)

1      90
2      61
3     153
4     422
5    9776
Name: star, dtype: int64


In [ ]:
# max_samples = 2500 # 최대 샘플 수
# oversampled_dfs = []
# for star, count in star_distribution.items():
#     subset = train[train['star'] == star]
#     if star != 5: # 별점 5점 제외하고 오버샘플링
#         resampled = resample(subset, replace=True, n_samples=max_samples, random_state=SEED)
#         oversampled_dfs.append(resampled)
#     else:
#         # shuffled_dfs = train[train['star'] == 5].sample(frac=1, random_state=2).reset_index(drop=True)
#         # subset = shuffled_dfs.iloc[:2500] # 별점이 5인 경우는 2500개 랜덤 추출해서 전체 비율 맞추기
#         oversampled_dfs.append(subset)

# train = pd.concat(oversampled_dfs).reset_index(drop=True)
# train['star'].value_counts().sort_index()

1    2500
2    2500
3    2500
4    2500
5    9776
Name: star, dtype: int64

In [ ]:
# # 별점 컬럼 삭제
# train = train.drop(columns=['star'])
# train.head()

,review,reply
0,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 ...,안녕하세요 1988응답하라추억의옛날도시락 입니다.\n\n주신 리뷰 잘 읽었습니다. ...
1,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 \n장난하는것도아니고,솔직한 리뷰 남겨 주셔서 감사합니다. \n\n저희 1988응답하라추억의옛날도시락은 ...
2,만들어 놓은지 한참 된 커피 ….배달 예정 시간 25분이 지났어요…….,안녕하세요 배달이 늦어져 죄송합니다. 한파로 길이 얼어서 배달기사님들이 상황이 여유...
3,밥이 쉬었어요... 단무지 때문도 아니고 김이 많아서 그런것도 아니고 쉬어서 냄새가...,밥이 쉬다뇨 말도 안되는 소리좀 하지마세요. 진짜 먼 정신이 이상한 사람인가 하루에...
4,잠실점에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와보니...,요기요가 주문어플에 렉이 자주 있어서.. 여러가지로 혼돈이 있어 정말 죄송하게 되었...


### 답변 맞춤법 처리

In [ ]:
# !git clone https://github.com/ssut/py-hanspell.git
# !cd py-hanspell
# !pip install py-hanspell/

In [ ]:
# from hanspell import spell_checker
# def correct_spell(text):
#     if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
#         corrected_text = spell_checker.check(text).checked
#         return corrected_text
#     else:
#         return text

# correct_spell(train['reply'][0])

In [ ]:
# # 한시간 반 걸림. 원한다면 click
# train['reply'] = train['reply'].apply(correct_spell)

### 가게명 키워드 컬럼화

In [ ]:
# ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]
# store_name = []
# for store in ygy_store_lst:
#     pat = re.split(r'[- &]', store)
#     pat = ''.join(pat)
#     store_name.append(pat)

# short_name = []
# for name in store_name:
#     for length in [2, 3, 4]:
#         cut = name[:length]
#         short_name.append(cut)
#     if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
#         cut = name[:6]
#         short_name.append(cut)

# short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
# short_name = list(set(short_name))
# print(len(short_name), short_name)

# # short_name 에서 불용어 처리
# drop_name = ['OK','갈비','감성','걸작','고기','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교','손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지', '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',]
# short_name = [col for col in short_name if col not in drop_name]
# print(len(short_name),short_name)

# train['store'] = ""
# for name in short_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '') # 각 가게 별로 컬럼 추가(그렇지 않으면 덮어쓰기 됨)
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # store 컬럼에 컬럼 값 합치기

# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 첫 번째 문자가 콤마인 경우 제거
# train = train.drop(columns=short_name) # 각 가게 별 컬럼 삭제
# train.head()

In [ ]:
# train['review'] = train['review'].astype(str)
# train['reply'] = train['reply'].astype(str)

- 여기까지 완료된 데이터 저장하기

In [ ]:
# # 전체
# # train.to_csv(f"{DATA_PATH}yogiyo_clean.csv", index=False)

# # 오버샘플
# train.to_csv(f"{DATA_PATH}yogiyo_OV_clean.csv", index=False)

- 여기까지 완료된 데이터 불러오기

In [ ]:
# # 전체
# train = pd.read_csv(f"{DATA_PATH}yogiyo_clean.csv")

# # 오버샘플
# train = pd.read_csv(f"{DATA_PATH}yogiyo_OV_clean.csv")

# review, reply = train["review"] , train["reply"]
# len(review) , len(reply)

(19776, 19776)

### 가게명 키워드 reply에서 삭제

In [ ]:
# # store 값이 있는 행만 가져오기
# train_del = train[train['store'].notna()]
# train_del

,review,reply,store
7,제 입맛에 안맞아요 뭔가 독특한 향신료 맛이에요 일반 닭갈비 맛이 아니에요 고기가 ...,이쁜 리뷰 감사합니다 고객님 즐거운 식사시간에 불편을 드려 죄송합니다 닭갈비 알밥이...,알촌
8,제 입맛에 안맞아요 뭔가 독특한 향신료 맛이에요 일반 닭갈비 맛이 아니에요 고기가 ...,이쁜 리뷰 감사합니다 고객님 즐거운 식사시간에 불편을 드려 죄송합니다 닭갈비 알밥이...,알촌
9,사장님 처럼 사장님만보기 기능있면좋겠지만만족하지만개선되었으면할때쓰고아니면안시키고말지...,저희 윤희 횟집을 찾아주셔서 감사드립니다 고객님께서 저희에게 충고해 주신 것에 대해...,윤희
14,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고,솔직한 리뷰 남겨 주셔서 감사합니다 저희 1988응답하라 추억의 옛날 도시락은 언제...,1988응답
15,사장님 처럼 사장님만보기 기능있면좋겠지만만족하지만개선되었으면할때쓰고아니면안시키고말지...,저희 윤희 횟집을 찾아주셔서 감사드립니다 고객님께서 저희에게 충고해 주신 것에 대해...,윤희
...,...,...,...
19338,앗 리뷰가 늦었네요 요즘처럼 고물가에 더욱 빛나는 곳 입니다 솔직히 저만 알고싶을 ...,아이고 이리 늘 찾아주시고 좋은 말씀들이 최가네를 춤추게 합니다 요즘 배달이든 마트...,"최가,최가네"
19351,최가네 한쌈음 한쌈은 항상 감동입니다 파김치 완전완전 세젤맛 먹어본 파김치중에 젤 ...,파김치가 입맛에 맞으셨다니 다행이네요 고객님께서 파를 너무 많이 보내주셔서 여러분하...,"최가,최가네"
19357,손 많이 가는 음식 푸짐히 먹고싶으신분 모두 주문하세요 후회 없습니다,아이고 이리 좋은 말씀이 최가네를 춤추게 하네요 앞으로도 고객님들 마음에 실망하시는...,"최가,최가네"
19381,후기 안쓰는데 처음후기 올려봅니다 다른분 후기중에 가격 올려야 되겠다는 댓글에 20...,글초 리뷰 이벤트도 없는데 이리들 귀한 시가 내셔서 좋은 글을 올려주시니 항상 너무...,"최가,최가네"


In [ ]:
# # store 를 리스트로 변환(반복문을 위해)
# train_del['store'] = train_del['store'].str.split(',')
# train_del['store']

<ipython-input-130-7ab0f5255dfd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_del['store'] = train_del['store'].str.split(',')


7             [알촌]
8             [알촌]
9             [윤희]
14        [1988응답]
15            [윤희]
           ...    
19338    [최가, 최가네]
19351    [최가, 최가네]
19357    [최가, 최가네]
19381    [최가, 최가네]
19404    [최가, 최가네]
Name: store, Length: 4487, dtype: object

In [ ]:
# del_reply = []
# for reply, store_words in zip(train_del['reply'], train_del['store']):
#     reply_words = reply.split()
#     new_reply_words = []
#     for reply_word in reply_words:
#         if not any(store_word in reply_word for store_word in store_words): # store_words에 있는 단어가 어떤 단어에도 포함되지 않으면 추가
#             new_reply_words.append(reply_word)

#     new_reply = ' '.join(new_reply_words)
#     del_reply.append(new_reply)

# train_del['reply'] = del_reply
# train_del['reply'][19404]

<ipython-input-134-99fbe3f86010>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_del['reply'] = del_reply


'맛있게 드셔주심에 저희가 너무 감사드립니다 항상 노력하는 될 수 있도록 할게요 좋은 글 남겨주셔서 감사드립니다'

In [ ]:
# # train 셋에 반영
# train[reply] = train_del['reply']
# train[reply][19404]

'맛있게 드셔주심에 저희가 너무 감사드립니다 항상 노력하는 될 수 있도록 할게요 좋은 글 남겨주셔서 감사드립니다'

### 지점명 reply에서 삭제

In [ ]:
# pattern = re.compile(r'\S+점')
# train["reply"].apply(lambda x: pattern.sub('', x)) #__점 제거

0        만족하셨더라면 좋았을걸 저희도 아쉽네요 전복 특에 우는 고명으로 전복 슬라이스가 올...
1        안녕하세요 고객님 다시 한번 사과의 말씀 드리고 싶습니다 오늘 직원들과의 미팅을 통...
2        죄송해요 고객님 최근 기상악화로 배달대행사에 애로사항이 많다 보니 1시간 2시간이 ...
3                            봉구 비어 닭 다리에요 매장에서 인기 많은데 죄송해요
4        안녕하세요 고객님 저희 매장 다시 찾아주셨는데 만족드리지 못해 죄송합니다 고의적으로...
                               ...                        
19771                        사진 잘 찍어주셔서 감사합니다 다음에 또 뵙겠습니다 
19772                                     맛있게 드셔주셔서 감사합니다 
19773           허겁지겁 맛있게 드셔주셔서 감사합니다 리뷰 감사합니다 다음에 또 뵙겠습니다 
19774    요청사항 언제든 환영입니다 맛있게 드셔주셔서 감사합니다 리뷰 감사합니다 다음에 또 ...
19775                                     헉 부산에서 처음 감사합니다 
Name: reply, Length: 19776, dtype: object

- 개별 삭제

In [ ]:
# train[train['reply'].str.contains("당신은 지금 치킨이 당긴다 헬리오 시티점입니다")].index

# remove_keywords = ["당신은 지금 치킨이 당긴다 헬리오 시티점입니다",]
# for keyword in remove_keywords:
#     train['reply'] = train['reply'].str.replace(keyword, '')

# train['reply'][2554]

Int64Index([ 2554,  2573,  2627,  2648,  2704,  2822,  2841,  2905,  3008,
             3204,
            ...
            10448, 10449, 10450, 10451, 10452, 10453, 10454, 10455, 10456,
            10457],
           dtype='int64', length=151)

- 여기까지 완료된 데이터 저장하기

In [ ]:
# # 전체
# # train.to_csv(f"{DATA_PATH}yogiyo_clean.csv", index=False)

# # 오버샘플 + 가게명 삭제
# train.to_csv(f"{DATA_PATH}yogiyo_OV_clean_store_del.csv", index=False)

### 여기까지 완료된 데이터 불러오기

In [ ]:
# 전체
# train = pd.read_csv(f"{DATA_PATH}yogiyo_clean.csv")

# 오버샘플 + 가게명 삭제
train = pd.read_csv(f"{DATA_PATH}yogiyo_OV_clean_store_del.csv")

train['review'] = train['review'].astype(str)
train['reply'] = train['reply'].astype(str)
review, reply = train["review"] , train["reply"]
len(review) , len(reply)

(19776, 19776)